# Problem Set 1: Informed Search

**Release Date:** 19 August 2022

**Due Date:** 23:59, 3 September 2022

## Overview

In class, we discussed a range of different searching algorithms. In this problem set, we get some hands-on practice by implementing them for simple logic problems. In particular, we investigate the Missionaries and Cannibals problem discussed in Tutorial 1, as well as a special 2-D Rubic’s Cube problem. Both problems operate in a fully- observable, single-agent, deterministic, episodic, static and discrete environments.

**References**: Lectures 2 & 3

**Required Files**:
* cube.csv
* utils.py
* requirements.txt

**Honour Code**: Note that plagiarism will not be condoned! You may discuss with your classmates and check the internet for references, but you MUST NOT submit code/report that is copied directly from other sources!

## Getting Started


This assignment as well as the subsequent ones are written in Python 3.9. You will need certain Python packages, which can be installed via terminal from your working directory using the following command:

```
    pip install -r requirements.txt
```

If you encounter any issues with the installation, please post them in the Forums, so that other students or instructors can help accordingly.
The template for all your tasks is provided in file search.py, with additional helper func- tions in cube.py and utils.py.

## Missionaries and Cannibals

In Tutorial 1, we discussed the Missionaries and Cannibals (MnC) problem, which is a classic river-crossing logic puzzle. It is not difficult to solve this problem by hand when we have 3 Missionaries and 3 Cannibals. However, the solution becomes less obvious when we generalize the problem to *m* Missionaries and *c* Cannibals where *m* and *c* are some positive integers. To solve this problem, we will implement both Breadth- First Search (BFS) and Depth-First Search (DFS). In the process, we hope that you will conceptually learn the differences in performance between these two search strategies.

The Missionaries and Cannibals problem is usually stated as follows: 3 missionaries and 3 cannibals are on one side of a river, along with a boat that can hold 1 or 2 people. Find a way to get everyone to the other side, without ever leaving a group of missionaries in one place outnumbered by the cannibals in that same place. You can try it https://javalab.org/en/boat_puzzle_en/ online to test your understanding of the problem. The following is an example of how to help 2 Missionaries and 2 Cannibals cross the river:

<pre>
<p style="text-align: center;">
Initial
M M | — — — — | _ _
| &lt;_ _&gt; — |
C C | — — — — | _ _


TWO_CANNIBAL
M M | - - - - | _ _
| &lt;C C&gt; - |
_ _ | - - - - | _ _

ONE_CANNIBAL
M M | - - - - | _ _
| - &lt;C _&gt; |
_ _ | - - - - | C _

TWO_MISSIONARY
_ _ | - - - - | _ _
| &lt;M M&gt; - |
C _ | - - - - | C _

ONE_MISSIONARY
_ _ | - - - - | M _
| - &lt;M _&gt; |
C _ | - - - - | C _

ONE_CANNIBAL_ONE_MISSIONARY
_ _ | - - - - | M _
| &lt;M C&gt; - |
_ _ | - - - - | C _

Goal
_ _ | - - - - | M M
| - &lt;_ _&gt; |
_ _ | - - - - | C C
</p>
</pre>

## Breadth-First Search (BFS) and Depth-First Search (DFS)

For this problem, all actions have the same cost (number of steps). BFS is an appropriate strategy to search all possible states. It first expands the root node, and then all the successors of the root node, and their successors. However, a BFS-based tree search algorithm will encounter a large number of repeated states in the tree and waste a lot of time in searching redundant nodes. On the other hand, BFS-based graph search algorithm will check for redundant paths to avoid visiting states more than once. To see the connection between tree search and graph search, you are tasked to implement both BFS tree search and BFS graph search.

DFS is another commonly seen algorithm in solving AI problems. It always expands the deepest node in the frontier first. DFS proceeds directly to the deepest level of the search tree. The search then “backs up” to the previous node that has unexpanded successors. Your task is to implement both DFS tree search and DFS graph search. You will observe that DFS-based tree search is susceptible to getting stuck in a looped path. In the context of this problem, it could mean keeping 2 Missionaries on the boat, rowing back-and-forth until exhaustion...

*Please run the following cell before proceeding. You may use any of the imported libraries/classes here. A few implementations have been provided in the included files. You may also wish to implment your own classes/helper functions. If you choose to do so, please also run the cell following the next.*

In [1]:
import os
import sys
import copy
import math
import heapq
import time
import timeout_decorator

import utils
import cube

# For following test cases
def wrap_test(func):
    def inner(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            return f'FAILED, reason: {str(e)}'
    return inner

In [4]:
""" ADD HELPER FUNCTION HERE """

"""
You CAN implement your own Node, PriorityQueue, Stack Class or use the provided 
ones
"""

class Node:
    r"""Node class for search tree
    Args:
        parent (Node): the parent node of this node in the tree
        act (Action): the action taken from parent to reach this node
        state (State): the state of this node
        g_n (float): the path cost of reaching this state
        h_n (float): the heuristic value of this state
    """
    
    def __init__(
            self, 
            parent: "Node", 
            act, 
            state, 
            g_n: float = 0.0, 
            h_n: float = 0.0):

        self.parent = parent # where am I from
        self.act = act # how to get here
        self.state = state # who am I
        self.g_n = g_n # what it takes to be here g(n)
        self.h_n = h_n # heuristic function h(n)
    
    def get_fn(self):
        r"""
        Returns the sum of heuristic and cost of the node
        """
        return self.g_n + self.h_n

    def __str__(self):
        return str(self.state)

    def __lt__(self, node):
        """Compare the path cost between states"""
        return self.g_n < node.g_n

    def __eq__(self, node):
        """Compare whether two nodes have the same state"""
        return isinstance(node, Node) and self.state == node.state

    def __hash__(self):
        """Node can be used as a KeyValue"""
        return hash(self.state)

class Stack:
    def __init__(self):
        self.stack = []
    
    def __len__(self):
        return len(self.stack)

    def __contains__(self, node):
        """Decide whether the node (state) is in the stack"""
        return any([item == node for item in self.stack])  

    def __repr__(self):
        return repr(self.stack)

    def push(self, node):
        self.stack.append(node)

    def pop(self):
        return self.stack.pop()
    
    def is_empty(self):
        return self.stack == []
    

class PriorityQueue:
    def __init__(self):
        self.heap = []

    def __contains__(self, node):
        """Decide whether the node (state) is in the queue"""
        return any([item == node for _, item in self.heap])

    def __delitem__(self, node):
        """Delete the an existing node in the queue"""
        try: 
            del self.heap[[item==node for _,item in self.heap].index(True)]
        except ValueError:
            raise KeyError(str(node)+"is not in the queue")
        heapq.heapify(self.heap) # O(n)

    def __getitem__(self, node):
        """Return the priority of the given node in the queue"""
        for value, item in self.heap:
            if item == node:
                return value
        raise KeyError(str(node)+"is not in the queue")

    def __len__(self):
        return len(self.heap)

    def push(self, priority, node):
        """Enqueue node with priority"""
        heapq.heappush(self.heap, (priority, node))

    def pop(self):
        """Dequeue node with highest priority (the minimum one)"""
        if self.heap:
            return heapq.heappop(self.heap)[1]
        else:
            raise Exception("Empty Priority Queue")

    def update(self, priority, node):
        """Update the node with the input priority"""
        self.__delitem__(node)
        self.push(priority, node)
    
    def get_priority(self, node):
        return self.__getitem__(node)
    
""" END HELPER FUNCTION"""

' END HELPER FUNCTION'

### Task 1: BFS-based Tree Search

Implement a **BFS-based tree** search as the function `bfs_tree_search(m,c)` that takes in the arguments `m` and `c`, where `m` is the number of Missionaries, and `c` is the number of Cannibals. `bfs_tree_search(m,c)` returns the loading (action) sequence of the boat on the river, for example, `[[0, 2], [0, 1], ...]` means that 2 Cannibals first take the boat to the other side of the river and then one Cannibal comes back alone, or `False` if there is no solution to the problem set up.

In [30]:
def bfs_tree_search(num_mis, num_can):
    r"""
    Breadth First Search (BFS) finds the solution to reach the goal from the 
    initial. By default, fail is True and returns False

    Tree search builds a child node for every reachable state, and push them 
    all into the froniter to be explored in the future. In every iteration, 
    agent explores a state with least priority (path cost). At the beginning 
    of search, agent can only explore from the initial state, and makes it as 
    the root node in the search tree.

    Solution should be the action taken from the root node (initial state) to 
    the leaf node (goal state) in the search tree.

    Args:
        num_mis: the number of the Missionaries 
        num_can: the number of the Cannibals

    Returns:
        solution (List): the action sequence
    """
    
    fail = True
    solution = []
    
    """ YOUR CODE HERE """
    # Need to track 2 things
    # - state tuple (m, c, boat), where m is missionary on start bank, c is cannibal on start bank,
    # and boat = 0 when boat at start and = 1 when boat at end
    # - action tuple (m, c), where m is missionary on boat, and c is cannibal on boat
    start_state_tuple = (num_mis, num_can, 0)
    goal_state = (0, 0, 1)
    # Generate all possible action tuples, regardless of m and c constraints
    # Boat can only hold 1 or 2 people
    possible_action_tuples = [[0, 1], [0, 2], [1, 0], [2, 0], [1, 1]]

    # frontier contains nodes --> tuples of state and solution to reach there
    start_node = utils.Node(None, None, start_state_tuple, 0, 0)
    frontier = utils.PriorityQueue()
    frontier.push(start_node.g_n, start_node)
    while len(frontier) != 0:
        iter += 1
        curr_node = frontier.pop()
        curr_state = curr_node.state
        # Check if state is goal state
        if curr_state == goal_state:
            trace_act = curr_node.act
            while trace_act != None:
                solution.append(trace_act)
                curr_node = curr_node.parent
                trace_act = curr_node.act
            fail = False
            solution.reverse()
            break
        multiplier = -1 if curr_node.state[2] == 0 else 1
        next_boat_pos = 1 if curr_node.state[2] == 0 else 0
        # Calculate all possible actions from this state
        for action in possible_action_tuples:
            # Calculate m and c on start side of river
            next_mis = curr_state[0] + action[0] * multiplier
            next_can = curr_state[1] + action[1] * multiplier
            if ((next_can > next_mis and next_mis > 0)
            or next_can < 0 or next_mis < 0
            or next_can > num_can or next_mis > num_mis):
                continue

            # Calculate m and c on other side of the river
            next_mis_other_bank = num_mis - next_mis
            next_can_other_bank = num_can - next_can
            if ((next_can_other_bank > next_mis_other_bank and next_mis_other_bank > 0)
            or next_can_other_bank < 0 or next_mis_other_bank < 0
            or next_can_other_bank > num_can or next_mis_other_bank > num_mis):
                continue

            # State falls within m and c constraints
            next_state = (next_mis, next_can, next_boat_pos)
            next_node = utils.Node(curr_node, action, next_state, curr_node.g_n + 1, 0)
            frontier.push(next_node.g_n, next_node)
    """ END YOUR CODE HERE """
    
    if fail:
        return False
    return solution

In [31]:
# Test cases for Task 1
@wrap_test
def test_bfs_tree_search(case):
    input_dict = case['input_dict']
    answer = case['answer']
    
    m, c = input_dict['initial']
    start = time.time()
    solution = bfs_tree_search(m, c)
    print(f"Time lapsed: {time.time() - start}")

    if solution is False:
        assert answer["solution"] is False, "Solution is not False"
    else:
        correctness = solution == answer["solution"]
        cost = len(solution)
        assert correctness, f"Fail to reach goal state with solution {solution}"
        assert cost <= answer['cost'], f"Cost is not optimal."
    return "PASSED"

mnc1 = {'input_dict': {"initial": [1, 0], "goal": [0, 0]}, 
        'answer': {"solution": [[1, 0]], "cost": 1}}
mnc2 = {'input_dict': {"initial": [0, 1], "goal": [0, 0]}, 
        'answer': {"solution": [[0, 1]], "cost": 1}}
mnc3 = {'input_dict': {"initial": [1, 1], "goal": [0, 0]}, 
        'answer': {"solution": [[1, 1]], "cost": 1}}

print(f'bfs_tree 1:  {test_bfs_tree_search(mnc1)}')
print(f'bfs_tree 2:  {test_bfs_tree_search(mnc2)}')
print(f'bfs_tree 3:  {test_bfs_tree_search(mnc3)}')

1
0
Time lapsed: 0.0
bfs_tree 1:  FAILED, reason: Solution is not False
1
0
Time lapsed: 0.0
bfs_tree 2:  FAILED, reason: Solution is not False
4
0
1
2
3
Time lapsed: 0.0
bfs_tree 3:  PASSED


### Task 2: DFS-based Tree Search
Implement the **DFS-based tree search** version of Task 1, `dfs_tree_search(m,c)`.

In [19]:
def dfs_tree_search(num_mis, num_can):
    r"""Deep First Search (DFS) finds the solution to reach the goal from the 
    initial. By default, fail is True and returns False

    DFS proceeds immediately to the deepest level of the search tree, where the 
    nodes have no successors. The search then “backs up” to the next deepest 
    node that still has unexpanded successors.
    
    Tree search builds a child node for every reachable state, and push them 
    all into a stack to be explored in the future. In every iteration, 
    agent explores the latest pushed state. At the beginning of search, agent 
    can only explore from the initial state, and makes it as the root node in 
    the search tree.

    Solution should be the action taken from the root node (initial state) to 
    the leaf node (goal state) in the search tree.

    Args:
        num_mis: the number of the Missionaries 
        num_can: the number of the Cannibals

    Returns:
        solution (List): the action sequence
    """

    fail = True
    solution = []
    
    """ YOUR CODE HERE """
    # Need to track 2 things
    # - state tuple (m, c, boat), where m is missionary on start bank, c is cannibal on start bank,
    # and boat = 0 when boat at start and = 1 when boat at end
    # - action tuple (m, c), where m is missionary on boat, and c is cannibal on boat
    start_state_tuple = (num_mis, num_can, 0)
    goal_state = (0, 0, 1)
    # Generate all possible action tuples, regardless of m and c constraints
    # Boat can only hold 1 or 2 people
    possible_action_tuples = [[0, 1], [0, 2], [1, 0], [2, 0], [1, 1]]

    # frontier contains nodes --> tuples of state and solution to reach there
    start_node = utils.Node(None, None, start_state_tuple, 0, 0)
    stack = utils.Stack()
    stack.push(start_node)
    
    while not stack.is_empty():
        curr_node = stack.pop()
        curr_state = curr_node.state
        # Check if state is goal state
        if curr_state == goal_state:
            trace_act = curr_node.act
            while trace_act != None:
                solution.append(trace_act)
                curr_node = curr_node.parent
                trace_act = curr_node.act
            fail = False
            solution.reverse()
            break
        multiplier = -1 if curr_node.state[2] == 0 else 1
        next_boat_pos = 1 if curr_node.state[2] == 0 else 0
        # Calculate all possible actions from this state and add to stack
        for action in possible_action_tuples:
            # Calculate m and c on start side of river
            next_mis = curr_state[0] + action[0] * multiplier
            next_can = curr_state[1] + action[1] * multiplier
            if ((next_can > next_mis and next_mis > 0)
            or next_can < 0 or next_mis < 0
            or next_can > num_can or next_mis > num_mis):
                continue

            # Calculate m and c on other side of the river
            next_mis_other_bank = num_mis - next_mis
            next_can_other_bank = num_can - next_can
            if ((next_can_other_bank > next_mis_other_bank and next_mis_other_bank > 0)
            or next_can_other_bank < 0 or next_mis_other_bank < 0
            or next_can_other_bank > num_can or next_mis_other_bank > num_mis):
                continue

            # State falls within m and c constraints
            next_state = (next_mis, next_can, next_boat_pos)
            next_node = utils.Node(curr_node, action, next_state, curr_node.g_n + 1, 0)
            stack.push(next_node)
    """ END YOUR CODE HERE """
    
    if fail:
        return False
    return solution

In [20]:
# Test cases for Task 2
@wrap_test
def test_dfs_tree_search(case):
    input_dict = case['input_dict']
    answer = case['answer']
    
    m, c = input_dict['initial']
    start = time.time()
    solution = dfs_tree_search(m, c)
    print(f"Time lapsed: {time.time() - start}")

    if solution is False:
        assert answer["solution"] is False, "Solution is not False"
    else:
        correctness = solution == answer["solution"]
        cost = len(solution)
        assert correctness, f"Fail to reach goal state with solution {solution}"
    return "PASSED"

mnc1 = {'input_dict': {"initial": [1, 0], "goal": [0, 0]}, 
        'answer': {"solution": [[1, 0]], "cost": 1}}
mnc2 = {'input_dict': {"initial": [0, 1], "goal": [0, 0]}, 
        'answer': {"solution": [[0, 1]], "cost": 1}}
mnc3 = {'input_dict': {"initial": [1, 1], "goal": [0, 0]}, 
        'answer': {"solution": [[1, 1]], "cost": 1}}

print(f'bfs_tree 1:  {test_dfs_tree_search(mnc1)}')
print(f'bfs_tree 2:  {test_dfs_tree_search(mnc2)}')
print(f'bfs_tree 3:  {test_dfs_tree_search(mnc3)}')

Time lapsed: 0.0
bfs_tree 1:  PASSED
Time lapsed: 0.0009083747863769531
bfs_tree 2:  PASSED
Time lapsed: 0.0
bfs_tree 3:  PASSED


### Task 3: BFS-based Graph Search

Implement the **BFS-based graph search** version of Task 1, `bfs_graph_search(m,c)`.

In [14]:
def bfs_graph_search(num_mis, num_can):
    r"""
    Breadth First Search finds the solution to reach the goal from the initial.
    By default, fail is True and returns False.

    Graph search requires to deal with the redundant path: cycle or loopy path.
    Modify the above implemented tree search algorithm to acceralate your AI.

    ***IMPORTANT***: make comments in the code you change and explain it.

    Args:
        num_mis: the number of the Missionaries 
        num_can: the number of the Cannibals

    Returns:
        solution (List): the action sequence
    """
    fail = True
    solution = []
    
    """ YOUR CODE HERE """
        # Need to track 2 things
    # - state tuple (m, c, boat), where m is missionary on start bank, c is cannibal on start bank,
    # and boat = 0 when boat at start and = 1 when boat at end
    # - action tuple (m, c), where m is missionary on boat, and c is cannibal on boat
    start_state_tuple = (num_mis, num_can, 0)
    goal_state = (0, 0, 1)
    # Generate all possible action tuples, regardless of m and c constraints
    # Boat can only hold 1 or 2 people
    possible_action_tuples = [[0, 1], [0, 2], [1, 0], [2, 0], [1, 1]]

    # frontier contains nodes --> tuples of state and solution to reach there
    start_node = utils.Node(None, None, start_state_tuple, 0, 0)
    frontier = utils.PriorityQueue()
    frontier.push(start_node.g_n, start_node)
    # ADDITION: Array to track visited nodes
    visited = set()
    visited.add(start_node)
    while len(frontier) != 0:
        curr_node = frontier.pop()
        curr_state = curr_node.state
        # Check if state is goal state
        if curr_state == goal_state:
            trace_act = curr_node.act
            while trace_act != None:
                solution.append(trace_act)
                curr_node = curr_node.parent
                trace_act = curr_node.act
            fail = False
            solution.reverse()
            break
        multiplier = -1 if curr_node.state[2] == 0 else 1
        next_boat_pos = 1 if curr_node.state[2] == 0 else 0
        # Calculate all possible actions from this state
        for action in possible_action_tuples:
            # Calculate m and c on start side of river
            next_mis = curr_state[0] + action[0] * multiplier
            next_can = curr_state[1] + action[1] * multiplier
            if ((next_can > next_mis and next_mis > 0)
            or next_can < 0 or next_mis < 0
            or next_can > num_can or next_mis > num_mis):
                continue

            # Calculate m and c on other side of the river
            next_mis_other_bank = num_mis - next_mis
            next_can_other_bank = num_can - next_can
            if ((next_can_other_bank > next_mis_other_bank and next_mis_other_bank > 0)
            or next_can_other_bank < 0 or next_mis_other_bank < 0
            or next_can_other_bank > num_can or next_mis_other_bank > num_mis):
                continue

            # State falls within m and c constraints
            next_state = (next_mis, next_can, next_boat_pos)
            # ADDITION: Check if this state has been visited
            next_visited = False
            for visited_node in visited:
                if next_state == visited_node.state:
                    next_visited = True
                    break
            
            # ADDITION: If state is visited, do not add it to frontier, else do add it to frontier
            if next_visited:
                continue
            next_node = utils.Node(curr_node, action, next_state, curr_node.g_n + 1, 0)
            frontier.push(next_node.g_n, next_node)
        # ADDITION: Add every processed node to the visited array for tracking purpose
        visited.add(curr_node)
    """ END YOUR CODE HERE """
    
    if fail:
        return False
    return solution

In [15]:
# Test cases for Task 3
@wrap_test
def test_bfs_graph_search(case):
    input_dict = case['input_dict']
    answer = case['answer']
    
    m, c = input_dict['initial']
    start = time.time()
    solution = bfs_graph_search(m, c)
    print(f"Time lapsed: {time.time() - start}")

    if solution is False:
        assert answer["solution"] is False, "Solution is not False"
    else:
        correctness = solution == answer["solution"]
        cost = len(solution)
        assert correctness, f"Fail to reach goal state with solution {solution}"
        assert cost <= answer['cost'], f"Cost is not optimal."
    return "PASSED"

mnc1 = {'input_dict': {"initial": [1, 0], "goal": [0, 0]}, 
        'answer': {"solution": [[1, 0]], "cost": 1}}
mnc2 = {'input_dict': {"initial": [0, 1], "goal": [0, 0]}, 
        'answer': {"solution": [[0, 1]], "cost": 1}}
mnc3 = {'input_dict': {"initial": [1, 1], "goal": [0, 0]}, 
        'answer': {"solution": [[1, 1]], "cost": 1}}

print(f'bfs_graph 1:  {test_bfs_graph_search(mnc1)}') 
print(f'bfs_graph 2:  {test_bfs_graph_search(mnc2)}') 
print(f'bfs_graph 3:  {test_bfs_graph_search(mnc3)}')

Time lapsed: 0.0
bfs_graph 1:  PASSED
Time lapsed: 0.0
bfs_graph 2:  PASSED
Time lapsed: 0.0
bfs_graph 3:  PASSED


### Task 4: Understanding performance differences
Find 3 problem configurations for which the algorithms you implemented in Tasks 1 to 3 will take in the order of seconds (or something “measureable” with a stopwatch) to run. Tabulate the results of at least 5 runs of each algorithm, and explain the differences in performance observed in the context of the structure of the underlying problem, Missionaries and Cannibals.


### Task 5: Generalizing the Problem
In the above problems, we assumed that the boat can take up to 2 passengers. In this task, generalize your solution to an MnC problem with `m` Missionaries, `c` Cannibals and a boat with capacity of up to `b` persons, where `1 ≤ b ≤ max(m, c)`. Implement a **DFS-based graph search** for this more general problem as a function `dfs_graph_search(m,c,b)`.

Include comments in the code to explain what modifications needed to be made to the earlier implementation in Task 2.

In [16]:
def dfs_graph_search(num_mis, num_can, boat_size):
    r"""Deep First Search (DFS) finds the solution to reach the goal from the 
    initial. By default, fail is True and returns an empty list [] as 
    solution.

    Graph search requires to deal with the redundant path: cycle or loopy path.
    Modify the above implemented tree search algorithm to acceralate your AI.

    ***IMPORTANT***: make comments in the code you change and explain it.

    Args:
        num_mis: the number of the Missionaries 
        num_can: the number of the Cannibals
        boat_size:: the number of people a boat can carry for one boat

    Returns:
        solution (List[Action]): the action sequence
    """


    fail = True
    solution = []
    
    """ YOUR CODE HERE """
    # Need to track 2 things
    # - state tuple (m, c, boat), where m is missionary on start bank, c is cannibal on start bank,
    # and boat = 0 when boat at start and = 1 when boat at end
    # - action tuple (m, c), where m is missionary on boat, and c is cannibal on boat
    start_state_tuple = (num_mis, num_can, 0)
    goal_state = (0, 0, 1)
    # Generate all possible action tuples, regardless of m and c constraints
    # Boat can only up to b people
    # ADDITION: Permutate through all possible actions given the m, c and boat_size constraints
    possible_action_arr = []
    for m in range(min(num_mis + 1, boat_size + 1)):
        remaining_capacity = boat_size - m if boat_size - m > 0 else 0
        for c in range(min(num_can + 1, remaining_capacity + 1)):
            action = [m, c]
            possible_action_arr.append(action)

    # frontier contains nodes --> tuples of state and solution to reach there
    start_node = utils.Node(None, None, start_state_tuple, 0, 0)
    stack = utils.Stack()
    stack.push(start_node)
    # ADDITION: Set to track visited nodes
    visited = set()
    visited.add(start_node)
    # ADDITION: Track shortest path to account for potential cycles with shorter paths
    shortest_path = math.inf
    goal_node_on_shortest_path = None
    while not stack.is_empty():
        curr_node = stack.pop()
        curr_state = curr_node.state
        # Check if state is goal state
        if curr_state == goal_state:
            # ADDITION: Check if path is longer than current shortest path, if it's shorter, update shortest path and goal node
            if curr_node.g_n >= shortest_path:
                continue
            shortest_path = curr_node.g_n
            goal_node_on_shortest_path = curr_node
            continue
        multiplier = -1 if curr_node.state[2] == 0 else 1
        next_boat_pos = 1 if curr_node.state[2] == 0 else 0
        # Calculate all possible actions from this state and add to stack
        for action in possible_action_arr:
            # Calculate m and c on start side of river
            next_mis = curr_state[0] + action[0] * multiplier
            next_can = curr_state[1] + action[1] * multiplier
            if ((next_can > next_mis and next_mis > 0)
            or next_can < 0 or next_mis < 0
            or next_can > num_can or next_mis > num_mis):
                continue

            # Calculate m and c on other side of the river
            next_mis_other_bank = num_mis - next_mis
            next_can_other_bank = num_can - next_can
            if ((next_can_other_bank > next_mis_other_bank and next_mis_other_bank > 0)
            or next_can_other_bank < 0 or next_mis_other_bank < 0
            or next_can_other_bank > num_can or next_mis_other_bank > num_mis):
                continue

            # State falls within m and c constraints
            next_state = (next_mis, next_can, next_boat_pos)
            # ADDITION: Check if this state has been visited
            next_visited = False
            for visited_node in visited:
                if next_state == visited_node.state:
                    next_visited = True
                    break
            
            # ADDITION: If state is visited, do not add it to stack, else do add it to stack
            if next_visited:
                continue
            next_node = utils.Node(curr_node, action, next_state, curr_node.g_n + 1, 0)
            stack.push(next_node)
        # ADDITION: Add every processed node to the visited set for tracking purpose
        visited.add(curr_node)

    # ADDITION: At the very end, check if a goal state was reached. If it was, trace the solution
    if goal_node_on_shortest_path != None:
        trace_act = goal_node_on_shortest_path.act
        curr_node = goal_node_on_shortest_path
        while trace_act != None:
            solution.append(trace_act)
            curr_node = curr_node.parent
            trace_act = curr_node.act
        fail = False
        solution.reverse()
    """ END YOUR CODE HERE """
    
    if fail:
        return False
    return solution

In [17]:
# Test cases for Task 5
@wrap_test
def test_dfs_graph_search(case):
    input_dict = case['input_dict']
    answer = case['answer']
    
    m, c = input_dict['initial']
    boat_size = input_dict['boat_size']
    start = time.time()
    solution = dfs_graph_search(m, c, boat_size)
    print(f"Time lapsed: {time.time() - start}")

    if solution is False:
        assert answer["solution"] is False, "Solution is not False"
    else:
        correctness = solution == answer["solution"]
        cost = len(solution)
        assert correctness, f"Fail to reach goal state with solution {solution}"
    return "PASSED"

mnc4 = {'input_dict': {"initial": [3, 0], "goal": [0, 0], "boat_size": 3}, 
        'answer': {"solution": [[3, 0]], "cost": 1}}
mnc5 = {'input_dict': {"initial": [0, 3], "goal": [0, 0], "boat_size": 3}, 
        'answer': {"solution": [[0, 3]], "cost": 1}}
mnc6 = {'input_dict': {"initial": [1, 1], "goal": [0, 0], "boat_size": 3}, 
        'answer': {"solution": [[1, 1]], "cost": 1}}
    
print(f'dfs_graph 1:  {test_dfs_graph_search(mnc4)}')
print(f'dfs_graph 2:  {test_dfs_graph_search(mnc5)}')
print(f'dfs_graph 3:  {test_dfs_graph_search(mnc6)}')

Time lapsed: 0.0009989738464355469
dfs_graph 1:  PASSED
Time lapsed: 0.0
dfs_graph 2:  PASSED
Time lapsed: 0.0
dfs_graph 3:  PASSED


## 2-D Rubik’s Cube
“The Rubik’s Cube is a 3-D combination puzzle invented in 1974 by Hungarian sculptor and professor of architecture Erno ̋ Rubik. Rubik’s Cube won the 1980 German Game of the Year special award for Best Puzzle. As of January 2009, 350 million cubes had been sold worldwide, making it the world’s bestselling puzzle game and bestselling toy.” – Wikipedia. In task 2, we explore a simplified version, 2-D Rubik’s “Cube”. To help you understand A* search, you will design and implement an A* search algorithm to find the solution of any cube.

Taking a cube of shape 3 rows × 3 columns as an example to explain the rule of the game. Given a cube of the initial state as follow, the rule of winning is to take as less as possible actions to reach the given goal state.

$$
\begin{align*}
initial:
\begin{bmatrix}
   R & G & B \\
   R & G & B \\
   R & G & B 
\end{bmatrix}
& \qquad goal:
\begin{bmatrix}
   R & R & R \\
   G & G & G \\
   B & B & B 
\end{bmatrix}
\end{align*}
$$

On each move, we can pick a [**number**] and a [**direction**] to change the cube, i.e. select a row and move left/right, or select a column and move up/down. Each move will only change the element in the selected row/column, the rest of the cube remains unchanged. For example, if row **1** moves **left**, every element in row 1 will take the position on its left and the head element will go to the tail of this row: [**RGB**] → [**GBR**]. It is clear that it is a recurrent move and consecutively moving left twice is the same as moving right once in a three-by-three cube. We encourage you to play with this cube to discover more insights and useful rules.

Here we provide a simple solution of the above cube example. You can walk through this solution step by step to get a better understanding of the cube.

$$
\begin{array}{rccccc}
& 0 & 1 & 2 & 3 & 4 \\
\begin{matrix}
         0 \\
         1 \\
         2 
      \end{matrix}
      & 
      \begin{bmatrix}
         R & G & B \\
         R & G & B \\
         R & G & B 
      \end{bmatrix}
      & 
      \begin{bmatrix}
         R & G & B \\
         G & B & R \\
         R & G & B 
      \end{bmatrix}
      & 
      \begin{bmatrix}
         R & G & B \\
         G & B & R \\
         B & R & G 
      \end{bmatrix}
      & 
      \begin{bmatrix}
         R & R & B \\
         G & G & R \\
         B & B & G 
      \end{bmatrix}
      & 
      \begin{bmatrix}
         R & R & R \\
         G & G & G \\
         B & B & B 
      \end{bmatrix} \\
      & (1, left) & (2, right) & (1, down) & (2, up) &
\end{array}
$$

### Task 6: A* Search
Implement the A* Search in two parts.
First, design your heuristic function `heuristic_func(problem, state)`, which takes in an instance of the cube problem class and a state. It returns the estimated cost of reaching the goal state from this state.
1. The heuristic function estimates the “distance” to the goal state.
2. The heuristic should be *admissible: never overestimates* the cost to reach a goal. With an admissible heuristic, A* is cost-optimal.
3. The template heuristic returns 0 for all the cases. It does not provide any information. Thus, you will see the connection between the A* search and the BFS graph search from the performance.
4. Try your best to find the best one.

In [11]:
def heuristic_func(problem: cube.Cube, state) -> float:
    r"""
    Computes the heuristic value of a state
    
    Args:
        problem (cube.Cube): the problem to compute
        state (cube.State): the state to be evaluated
        
    Returns:
        h_n (float): the heuristic value 
    """
    h_n = 0.0
    goals = problem.goal

    """ YOUR CODE HERE """
    # Calculate how many tiles are not in the right place, and divide
    # that number by the max(rows, columns)
    shape = goals.shape
    max_dim = -1
    for dim in shape:
        if dim <= max_dim:
            continue
        max_dim = dim

    for i in len(goals.layout):
        if goals.layout[i] == state.layout[i]:
            continue
        h_n += 1

    h_n /= max_dim
    print(h_n)
    """ END YOUR CODE HERE """

    return h_n

Second, implement the A* search astar_search(problem), which takes in an instance of the Cube class (see below), and returns a sequence of actions from the provided action set.

1. A* search is a best-first search that uses the evaluation function

    `f (state) = g(state) + h(state)`

    to estimate the cost of the best path that continues from state to goal state.

2. A* search should be aware of whether a new state has been reached or not
3. A* search should explore the node with the lowest possible cost to the goal state on each step
4. In the event that a better path to an unexplored state is found, A* search should update its information in the “waiting list”.

If there is no set of actions that can bring the initial configuration to the goal state, `astar_search(problem)` should return `False`.

*Hint: It might be useful to create additional functions for PriorityQueue class. Place these functions in the cells above the description of [Task 1](#task-1-bfs-based-tree-search).*

In [12]:
def astar_search(problem: cube.Cube):
    r"""
    A* Search finds the solution to reach the goal from the initial.
    By default, fail is True and returns False.
    
    Args:
        problem (cube.Cube): Cube instance

    Returns:
        solution (List[Action]): the action sequence
    """
    fail = True
    solution = []
    
    """ YOUR CODE HERE """
    start_node = utils.Node(None, None, problem.initial, 0, heuristic_func(problem, problem.initial))
    frontier = utils.PriorityQueue()
    frontier.push(start_node.get_fn(), start_node)
    visited = []
    visited.append(start_node)
    shortest_path = math.inf
    goal_node_on_shortest_path = None
    while len(frontier) != 0:
        curr_node = frontier.pop()
        curr_state = curr_node.state
        # Check if state is goal state
        if problem.goal_test(curr_state):
            if curr_node.get_fn() >= shortest_path:
                continue
            shortest_path = curr_node.get_fn()
            goal_node_on_shortest_path = curr_node
            continue
        multiplier = -1 if curr_node.state[2] == 0 else 1
        next_boat_pos = 1 if curr_node.state[2] == 0 else 0
        # Calculate all possible actions from this state
        for action in problem.actions(curr_state):
            next_state = problem.result(curr_state, action)
            next_visited = False
            for visited_node in visited:
                if next_state == visited_node.state:
                    next_visited = True
                    break
            if next_visited:
                continue
            next_node = utils.Node(curr_node, action, next_state,
                problem.path_cost(curr_node.g_n, curr_state, action, next_state), heuristic_func(problem, next_state))
            frontier.push(next_node.get_fn(), next_node)
        visited.append(curr_node)

    if goal_node_on_shortest_path != None:
        trace_act = goal_node_on_shortest_path.act
        curr_node = goal_node_on_shortest_path
        while trace_act != None:
            solution.append(trace_act)
            curr_node = curr_node.parent
            trace_act = curr_node.act
        fail = False
        solution.reverse()
    """ END YOUR CODE HERE """
    
    if fail:
        return False
    return solution

In [13]:
# Test cases for Task 6
@wrap_test
def test_astar(case):

    input_dict = case['input_dict']
    answer = case['answer']
    problem = cube.Cube(input_dict = input_dict)
    
    start = time.time()
    solution = astar_search(problem)
    print(f"Time lapsed: {time.time() - start}")

    if solution is False:
        assert answer["solution"] is False, "Solution is not False"
    else:
        correctness, cost = problem.verify_solution(solution, _print=False)
        assert correctness, f"Fail to reach goal state with solution {solution}"
        assert cost <= answer['cost'], f"Cost is not optimal."
    return "PASSED"

cube1 = {'input_dict': {"initial": {'shape': [3, 3], 'layout': ['N', 'U',   
    'S', 'N','U', 'S', 'N', 'U', 'S']}, "goal": {'shape': [3, 3], 'layout': 
    ['N', 'U', 'S', 'N', 'U', 'S', 'N', 'U', 'S']}}, 'answer': {"solution": 
    [], "cost": 0}}

cube2 = {'input_dict': {"initial": {'shape': [3, 3], 'layout': ['S', 'O', 
    'C', 'S', 'O', 'C', 'S', 'O', 'C']}, "goal": {'shape': [3, 3], 
    'layout': ['S', 'S', 'S', 'O', 'O', 'O', 'C', 'C', 'C']}}, 'answer': 
    {"solution": [[2, 'right'], [1, 'left'], [1, 'down'], 
    [2, 'up']], "cost": 4}}

cube3 = {'input_dict': {"initial": {'shape': [3, 3], 'layout': ['N', 'U',   
    'S', 'N','U', 'S', 'N', 'U', 'S']}, "goal": {'shape': [3, 3], 'layout': 
    ['S', 'U', 'N', 'S', 'U', 'N', 'S', 'U', 'N']}}, 'answer': {"solution": 
    [[0, 'right'], [2, 'left'], [2, 'down'], [1, 'up'], [1, 'left'], 
    [2, 'right']], "cost": 6}}

cube4 = {'input_dict':{"initial": {'shape': [3, 4], 'layout': [1, 1, 9, 0,
    2, 2, 0, 2, 9, 0, 1, 9]}, "goal": {'shape': [3, 4], 'layout': [2, 1, 0,
    9, 2, 1, 0, 9, 2, 1, 0, 9]}}, 'answer': {"solution": [[3, "up"], 
    [1, "down"], [2, "left"], [0, "right"]], "cost": 4}}

print('cube1: ' + test_astar(cube1))
print('cube2: ' + test_astar(cube2))
print('cube3: ' + test_astar(cube3))
print('cube4: ' + test_astar(cube4))

cube1: FAILED, reason: 'int' object is not iterable
cube2: FAILED, reason: 'int' object is not iterable
cube3: FAILED, reason: 'int' object is not iterable
cube4: FAILED, reason: 'int' object is not iterable


### Task 7: Consistency & Admissibility
Explain why the heuristic you designed for Task 6 is *consistent* and *admissible*.

## Helper Code
To allow you to focus on implementing search instead of having to set up state, the class `Cube` provided in cube.py supports the following methods:

- `goal_test(state)`: tests whether the provided `state` is the goal state.

- `actions(state)`: returns a list of actions at the provided `state`.

- `result(state, action)`: returns the new state after taking `action` from the provide `state`. It is deterministic.

- `path_cost(c, state1, action, state2)`: returns the accumulated cost of reaching state1 from the initial state and then reaching state2 from state1 by action.

In the cube problem, the state is an instance of `State` class. It is a hashable type. `Action` in `Cube` is a tuple of an integer representing label and a string representing direction. Your search function should take and only take legal actions to transition to another state.

For your convenience, we have provided a `Node` class for constructing a search tree, a `Stack` class, and a `PriorityQueue` class for your search algorithm in the `utils.py`. You may also choose to implement your own `Node`, `Stack`, and `PriorityQueue` class instead. Our autograder will follow the same import structure as that of the `search.py`.

**If you choose to override the provided helpers, please include all your code implementations in the template file `search.py` as well as Coursemology .**

## Test Cases

To help with your implementation, we have provided some examples as test cases. These not sufficient to ensure that your code is works correctly, and we encourage you to write your own additional test cases to test and debug your code.

Note that your answers may be slightly different from the answers provided since mul- tiple valid solutions sharing the same cost may exist. During grading, your code will be evaluated on hidden test cases on top of the ones we have provided. We will validate your solution and compare the resulting cost to the expected optimal cost.

Note that we will have hidden test case(s) to check the quality of your heuristic function in the A* search algorithm. Basically, a good heuristic function should provide valuable information to the search, and thus it reduces the number of explorations before finding the best solution. You can keep track of the size of the “reached” state to help you design a better heuristic.

<center><i>Have fun and enjoy coding.<i><center>

# Submission

Once you are done, please submit your work to Coursemology, by copying the right snippets of code into the corresponding box that says 'Your answer', and click 'Save'.  After you save, you can make changes to your
submission.

Once you are satisfied with what you have uploaded, click 'Finalize submission.'  **Note that once your submission is finalized, it is considered to be submitted for grading and cannot be changed**. If you need to undo
this action, you will have to email your assigned tutor for help. Please do not finalize your submission until you are sure that you want to submit your solutions for grading. 
